In [ ]:
import  numpy as np
import pandas as pd

In [ ]:
import string
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
import re
from nltk.stem import PorterStemmer

In [ ]:
import statsmodels.api as sm
from collections import Counter
from gensim.models import word2vec

In [ ]:
data = pd.read_csv('../input/ziphack/train_file.csv')
test = pd.read_csv('../input/ziphack/test_file.csv')

In [ ]:
data["MaterialType"] = data['MaterialType'].str.replace('[^\w\s]','')

In [ ]:
data.drop(['UsageClass','CheckoutType','CheckoutYear','CheckoutMonth'],inplace=True,axis=1)
test.drop(['UsageClass','CheckoutType','CheckoutYear','CheckoutMonth'],inplace=True,axis=1)

In [ ]:
data.drop(['Checkouts','PublicationYear'],inplace = True,axis = 1)
test.drop(['Checkouts','PublicationYear'],inplace = True,axis = 1)

In [ ]:
data.head()

In [ ]:
y = data["MaterialType"]

In [ ]:
data = data.apply(lambda x: x.astype(str).str.lower())
test = test.apply(lambda x: x.astype(str).str.lower())

In [ ]:
data["Title"] = data['Title'].str.replace('[^\w\s]','')
test["Title"] = test['Title'].str.replace('[^\w\s]','')
data["Subjects"] = data['Subjects'].str.replace('[^\w\s]','')
test["Subjects"] = test['Subjects'].str.replace('[^\w\s]','')
##test["Subjects"] = test['Subjects'].str.replace('[0-9]','')

In [ ]:
from nltk.corpus import stopwords
sw = stopwords.words('english')

In [ ]:
sw = stopwords.words('english')

In [ ]:
def stopwords(text):
    text = [word.lower() for word in text.split() if word.lower() not in sw]
    return " ".join(text)

In [ ]:
data['Title'] = data['Title'].apply(stopwords)
data['Subjects'] = data['Subjects'].apply(stopwords)
test['Title'] = test['Title'].apply(stopwords)
test['Subjects'] = test['Subjects'].apply(stopwords)

In [ ]:
#print(data.nunique())

In [ ]:
import nltk

w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()
stemmer = PorterStemmer()

In [ ]:
def pre(data):
    word = lemmatizer.lemmatize(data) 
    return word

In [ ]:
data['Title'] = data['Title'].apply(pre)
data['Subjects'] = data['Subjects'].apply(pre)
test['Subjects'] = test['Subjects'].apply(pre)
test['Title'] = test['Title'].apply(pre)

In [ ]:
data["Subjects"] = data['Subjects'].replace('nan','')
test["Subjects"] = test['Subjects'].replace('nan','')

In [ ]:
data.head()

In [ ]:
data["Final"] = data["Title"].map(str) + ' ' + data["Subjects"]
test["Final"] = test["Title"].map(str) + ' ' + test["Subjects"]

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn import preprocessing as skp
label_encoder = LabelEncoder()

In [ ]:
led = LabelEncoder()
led.fit(data["MaterialType"])
data["MaterialType"] = led.transform(data["MaterialType"].values)

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import tensorflow as tf
from keras import backend
import logging

In [ ]:
from keras.preprocessing import text, sequence
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences


vocab_size = 20000  # based on words in the entire corpus
max_len = 90        # based on word count in phrases

all_corpus   = list(data['Final'].values) + list(test['Final'].values)
train_phrases  = list(data['Final'].values) 
test_phrases   = list(test['Final'].values)
X_train_target_binary = pd.get_dummies(data['MaterialType'])

#Vocabulary-Indexing of thetrain and test phrases, make sure "filters" parm doesn't clean out punctuations

tokenizer = Tokenizer(num_words=vocab_size, lower=True, filters='\n\t')
tokenizer.fit_on_texts(all_corpus)
encoded_train_phrases = tokenizer.texts_to_sequences(train_phrases)
encoded_test_phrases = tokenizer.texts_to_sequences(test_phrases)

#Watch for a POST padding, as opposed to the default PRE padding

X_train_words = sequence.pad_sequences(encoded_train_phrases, maxlen=max_len,  padding='post')
X_test_words = sequence.pad_sequences(encoded_test_phrases, maxlen=max_len,  padding='post')
print (X_train_words.shape)
print (X_test_words.shape)
print (X_train_target_binary.shape)

print ('Done Tokenizing and indexing phrases based on the vocabulary learned from the entire Train and Test corpus')


In [ ]:
data.groupby('MaterialType')['ID'].agg('count').reset_index()

In [ ]:
save_test = pd.merge(test, data[["Final", "MaterialType"]], on="Final", how="inner")
print ("Number of overlapping phrases  ", save_test.shape[0])
#print ("% of neutral sentiment phrases",save_test[(save_test['Sentiment'] == 2)].count()[0] /save_test.shape[0])

In [ ]:
word_index = tokenizer.word_index
embeddings_index = {}
embedding_size = 300
EMBEDDING_FILE =  '../input/fasttext-crawl-300d-2m/crawl-300d-2M.vec'

with open(EMBEDDING_FILE, 'r') as f:
    for line in f:
        values = line.rstrip().rsplit(' ')
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
        
num_words = min(vocab_size, len(word_index) + 1)
embedding_matrix = np.zeros((num_words, embedding_size))
for word, i in word_index.items():
    if i >= vocab_size:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector
        
logging.info('Done building embedding matrix from FastText') 

In [ ]:
from keras.callbacks import EarlyStopping
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, Conv1D, CuDNNGRU, CuDNNLSTM, BatchNormalization
from keras.layers import  GlobalMaxPool1D, SpatialDropout1D
from keras.layers import Bidirectional
from keras.models import Model
from keras import optimizers

early_stop = EarlyStopping(monitor = "val_loss", mode="min", patience = 3, verbose=1)

print("Building layers")        
nb_epoch = 25
print('starting to stitch and compile  model')

# Embedding layer for text inputs
input_words = Input((max_len, ))
x_words = Embedding(num_words, embedding_size,weights=[embedding_matrix],trainable=False)(input_words)
x_words = SpatialDropout1D(0.2)(x_words)
x_words = Bidirectional(CuDNNGRU(50, return_sequences=True))(x_words)
x_words = Dropout(0.2)(x_words)
x_words = Conv1D(128, 1, strides = 1,  padding='causal', activation='relu', )(x_words)
x_words = Conv1D(256, 3, strides = 1,  padding='causal', activation='relu', )(x_words)
x_words = Conv1D(512, 5, strides = 1,   padding='causal', activation='relu', )(x_words)
x_words = GlobalMaxPool1D()(x_words)
x_words = Dropout(0.2)(x_words)

x = Dense(50, activation="relu")(x_words)
x = Dropout(0.2)(x)
predictions = Dense(8, activation="softmax")(x)

model = Model(inputs=[input_words], outputs=predictions)
model.compile(optimizer='rmsprop' ,loss='categorical_crossentropy', metrics=['accuracy'])
print(model.summary())


print("OOV word count:", len(set(word_index) - set(embeddings_index)))

In [ ]:
history = model.fit([X_train_words], X_train_target_binary, epochs=nb_epoch, verbose=1, batch_size = 512, callbacks=[early_stop], validation_split = 0.05, shuffle=True)
#history = model.fit(X_train_words, X_train_target_binary, epochs=10, verbose=1, batch_size = 512,  validation_split = 0.1, shuffle=True)
train_loss = np.mean(history.history['loss'])
val_loss = np.mean(history.history['val_loss'])
print('Train loss: %f' % (train_loss*100))
print('Validation loss: %f' % (val_loss*100))

In [ ]:
pred_test = model.predict([X_test_words], batch_size=1024, verbose = 0)
print (pred_test.shape)

In [ ]:
max_pred = np.round(np.argmax(pred_test, axis=1)).astype(int)
submission = pd.DataFrame({'ID':test['ID'],'MaterialType': max_pred})

In [ ]:
submission.shape

In [ ]:
submission["MaterialType"] = led.inverse_transform(submission["MaterialType"])
submission = submission.apply(lambda x: x.astype(str).str.upper())
submission.to_csv('submission_cnn.csv', index=False)